In [16]:
import json
#import _cvxcore
#import cvxpy
import fancyimpute
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler,SimpleFill
import numpy as np
import pandas as pd
import random
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pickle
from argparse import ArgumentParser
import math

In [31]:
ARG_PARSER = ArgumentParser()

ARG_PARSER.add_argument('--air', default=True)
ARG_PARSER.add_argument('--mimic', default=False)
ARG_PARSER.add_argument('--ehr', default=False)

ARG_PARSER.add_argument('--pred', default=False)
ARG_PARSER.add_argument('--imp', default=True)

ARG_PARSER.add_argument('--pred_len', default=8, type=int)
ARG_PARSER.add_argument('--missingRate', default=10, type=int)

_StoreAction(option_strings=['--imp'], dest='imp', nargs=None, const=None, default=10, type=<class 'int'>, choices=None, help=None, metavar=None)

In [32]:
def get_loss(X, X_pred, Y):
    # find ones in Y but not in X (ground truth)
    mask = np.isnan(X) ^ np.isnan(Y)

    X_pred = np.nan_to_num(X_pred)
    pred = X_pred[mask]
    label = Y[mask]

    mae = np.abs(pred - label).sum() / (1e-5 + np.sum(mask))
    mre = np.abs(pred - label).sum() / (1e-5 + np.sum(np.abs(label)))

    return {'mae': mae, 'mre': mre}

In [33]:
def dataPrep(data,flag):
    data=data.sort_values(by=['person_id','Age'])
    del data['person_id']
    del data['visit_start_datetime']
    del data['visit_end_datetime']
    del data['val']
    del data['Interval']
    #print(data['Age'].head())
    #print(data.columns)
    if flag==1:
        del data['interval']
        #del data['intervalReverse']
    data.rename({'MEAS_3038553': 'BMI'}, axis=1, inplace=True)
    #print(data.shape)
    data = data.to_numpy()
    data = np.reshape(data, (int(data.shape[0]/20), 20, data.shape[1]))
    #data=data[2500:3000,:,:]
    #print(data.shape)
    return data

In [34]:
args = ARG_PARSER.parse_args(args=[])

In [ ]:
def imp(data,mask,missingRate):
    data = data.to_numpy()
    data = np.reshape(data, (int(data.shape[0]/20), 20, data.shape[1]))

    mask = mask.to_numpy()
    mask = np.reshape(mask, (int(mask.shape[0]/20), 20, mask.shape[1]))

    mask=mask[:,:,0]
    #print(mask)
    y=data[:,:,2].copy()
    testMask=mask.copy()

    samples=0
    pids=0

    for i in range(data.shape[0]):
        idxs = np.nonzero(mask[i,:] == 1)
        idxs=(np.array(idxs)).reshape(-1)
        #print(idxs)
        #idxs=np.random.choice(idxs,3)
        #print(len(idxs))
        samples=samples+len(idxs)

        if(missingRate==50):
            if(len(idxs)>4):
                idxs=np.random.choice(idxs,5)
                data[i,idxs[0],2]=0
                data[i,idxs[1],2]=0
                data[i,idxs[2],2]=0
                data[i,idxs[3],2]=0
                data[i,idxs[4],2]=0
                #print(mask[i,:])
                mask[i,idxs[0]]=0
                mask[i,idxs[1]]=0
                mask[i,idxs[2]]=0
                mask[i,idxs[3]]=0
                mask[i,idxs[4]]=0
                pids=pids + 5
                break;
        if(missingRate>=40):
            if(len(idxs)>3):
                idxs=np.random.choice(idxs,4)
                data[i,idxs[0],2]=0
                data[i,idxs[1],2]=0
                data[i,idxs[2],2]=0
                data[i,idxs[3],2]=0
                #print(mask[i,:])
                mask[i,idxs[0]]=0
                mask[i,idxs[1]]=0
                mask[i,idxs[2]]=0
                mask[i,idxs[3]]=0
                pids=pids + 4
                break;
        if(missingRate>=30):
            if(len(idxs)>2):
                idxs=np.random.choice(idxs,3)
                data[i,idxs[0],2]=0
                data[i,idxs[1],2]=0
                data[i,idxs[2],2]=0
                #print(mask[i,:])
                mask[i,idxs[0]]=0
                mask[i,idxs[1]]=0
                mask[i,idxs[2]]=0
                pids=pids + 3
                break;
        if(missingRate>=20):
            if(len(idxs)>1):
                idxs=np.random.choice(idxs,2)
                data[i,idxs[0],2]=0
                data[i,idxs[1],2]=0
                #print(mask[i,:])
                mask[i,idxs[0]]=0
                mask[i,idxs[1]]=0
                pids=pids + 2
                break;
        if(missingRate>=10):
            if(len(idxs)>0):
                idxs=np.random.choice(idxs,1)
                data[i,idxs,2]=0
                mask[i,idxs]=0
                pids=pids + 1
        testMask[i,:]=testMask[i,:]-mask[i,:]
        y[i,:]=y[i,:]*testMask[i,:]

    data=data[:,:,2]
    #     data=data.transpose()
    #     mask=mask.transpose()
    #     y=y.transpose()
    #     testMask=testMask.transpose()
    data[data==0]=np.nan
    data=pd.DataFrame(data)

    data_complete=data.fillna(data.mean())

    data=data.fillna(0)
    data_complete=data_complete.fillna(0)

    data = data.to_numpy()
    data_complete = data_complete.to_numpy()

    data_complete=np.multiply(data_complete,testMask)

    #print(data_complete[0,:])
    #print(y[0,:])
    #print(testMask[0,:])
    mae = np.abs(data_complete - y).sum() / (1e-5 + np.sum(testMask))
    mre = np.abs(data_complete - y).sum() / (1e-5 + np.sum(np.abs(y)))

    outBmi=data_complete[np.nonzero(testMask)]
    inBmi=y[np.nonzero(testMask)]
    #     with open('results/mean/'+str(m) +'/outBmiImp', 'wb') as fp:
    #         pickle.dump(outBmi, fp)
    #     with open('results/mean/'+str(m) +'/inBmiImp', 'wb') as fp:
    #         pickle.dump(inBmi, fp)
    #print(len(inBmi))
    #print(len(outBmi))
    #plotBmi(inBmi,outBmi,len(inBmi))

    loss = (outBmi - inBmi)
    loss=np.asarray([abs(number) for number in loss])
    variance = sum([((x - mae) ** 2) for x in loss]) / len(loss) 
    res = variance ** 0.5
    ci=1.96*(res/(math.sqrt(len(loss))))
    print("CI",ci)

    print("Mean MAE: ",mae)
    #print("Mean MRE: ",mre)

In [ ]:
def pred(data,mask,predWin):
    data = data.to_numpy()
    data = np.reshape(data, (int(data.shape[0]/20), 20, data.shape[1]))

    mask = mask.to_numpy()
    mask = np.reshape(mask, (int(mask.shape[0]/20), 20, mask.shape[1]))

    mask=mask[:,:,0]
    y=data[:,:,2].copy()
    testMask=mask.copy()

    if args.pred==2:
        for i in range(data.shape[0]):
            #if(data[i,])
            j=20
            if(predWin==8):
                k=16
            elif(predWin==7):
                k=14
            elif(predWin==6):
                k=12
            elif(predWin==5):
                k=10
            data[i,j-k:j,:]=0
            mask[i,j-k:j]=0
            y[i,0:j-k]=0
            testMask[i,0:j-k]=0

    data=data[:,:,2]
    #     data=data.transpose()
    #     mask=mask.transpose()
    #     y=y.transpose()
    #     testMask=testMask.transpose()
    data[data==0]=np.nan
    data=pd.DataFrame(data)

    data_complete=data.fillna(data.mean())

    data=data.fillna(0)
    data_complete=data_complete.fillna(0)

    data = data.to_numpy()
    data_complete = data_complete.to_numpy()

    data_complete=np.multiply(data_complete,testMask)
    #     print(data_complete[0,:])
    #     print(y[0,:])
    #     print(testMask[0,:])

    mae = np.abs(data_complete - y).sum() / (1e-5 + np.sum(testMask))
    mre = np.abs(data_complete - y).sum() / (1e-5 + np.sum(np.abs(y)))

    outBmi=data_complete[np.nonzero(testMask)]
    inBmi=y[np.nonzero(testMask)]

    #     with open('results/mean/'+str(m) +'/outBmiPred2', 'wb') as fp:
    #         pickle.dump(outBmi, fp)
    #     with open('results/mean/'+str(m) +'/inBmiPred2', 'wb') as fp:
    #         pickle.dump(inBmi, fp)

    #plotBmi(inBmi,outBmi,len(inBmi))
    loss = (outBmi - inBmi)
    loss=np.asarray([abs(number) for number in loss])
    variance = sum([((x - mae) ** 2) for x in loss]) / len(loss) 
    res = variance ** 0.5
    ci=1.96*(res/(math.sqrt(len(loss))))
    print("CI",ci)

    print("Mean MAE: ",mae)
    print("Mean MRE: ",mre)

In [ ]:
def ehrPred(predWin):

    data=pd.read_csv('.../aaai/data/ehr/preprocess/test/finalTest102.csv',header=0)
    mask=pd.read_csv('.../aaai/data/ehr/preprocess/test/mask/maskTest102.csv',header=0)

    data=dataPrep(data,0)
    mask=dataPrep(mask,1)
    mask=mask[:,:,316]
    y=data[:,:,316].copy()
    testMask=mask.copy()

    for i in range(data.shape[0]):
        #if(data[i,])
        j=20
        if(predWin==8):
            k=16
        elif(predWin==7):
            k=14
        elif(predWin==6):
            k=12
        elif(predWin==5):
            k=10

        data[i,j-k:j,:]=0
        mask[i,j-k:j]=0
        y[i,0:j-k]=0
        testMask[i,0:j-k]=0

    data=data[:,:,316]
    data=data.transpose()
    mask=mask.transpose()
    y=y.transpose()
    testMask=testMask.transpose()
    data[data==0]=np.nan
    data=pd.DataFrame(data)

    data_complete=data.fillna(data.mean())

    data=data.fillna(0)
    data_complete=data_complete.fillna(0)

    data = data.to_numpy()
    data_complete = data_complete.to_numpy()

    data_complete=np.multiply(data_complete,testMask)
    #     print(data_complete[0,:])
    #     print(y[0,:])
    #     print(testMask[0,:])

    mae = np.abs(data_complete - y).sum() / (1e-5 + np.sum(testMask))
    mre = np.abs(data_complete - y).sum() / (1e-5 + np.sum(np.abs(y)))

    outBmi=data_complete[np.nonzero(testMask)]
    inBmi=y[np.nonzero(testMask)]

    #     with open('results/mean/'+str(m) +'/outBmiPred2', 'wb') as fp:
    #         pickle.dump(outBmi, fp)
    #     with open('results/mean/'+str(m) +'/inBmiPred2', 'wb') as fp:
    #         pickle.dump(inBmi, fp)

    #plotBmi(inBmi,outBmi,len(inBmi))

    print("Mean MAE: ",mae)
    print("Mean MRE: ",mre)

In [ ]:
def ehrImpute(missingRate):
    for m in range(1,6):
    data=pd.read_csv('.../aaai/data/ehr/preprocess/test/finalTest102.csv',header=0)
    mask=pd.read_csv('.../aaai/data/ehr/preprocess/test/mask/maskTest102.csv',header=0)

    data=dataPrep(data,0)
    mask=dataPrep(mask,1)
    mask=mask[:,:,316]
    y=data[:,:,316].copy()
    testMask=mask.copy()
    
    samples=0
    pids=0

    for i in range(data.shape[0]):
        idxs = np.nonzero(mask[i,:] == 1)
        idxs=(np.array(idxs)).reshape(-1)
        #print(idxs)
        #idxs=np.random.choice(idxs,3)
        #print(len(idxs))
        
        samples=samples+len(idxs)
        if(missingRate==50):
            if(len(idxs)>4):
                idxs=np.random.choice(idxs,5)
                data[i,idxs[0],316]=0
                data[i,idxs[1],316]=0
                data[i,idxs[2],316]=0
                data[i,idxs[3],316]=0
                data[i,idxs[4],316]=0
                #print(mask[i,:])
                mask[i,idxs[0]]=0
                mask[i,idxs[1]]=0
                mask[i,idxs[2]]=0
                mask[i,idxs[3]]=0
                mask[i,idxs[4]]=0
                pids=pids + 5
                break;
        if(missingRate>=40):
            if(len(idxs)>3):
                idxs=np.random.choice(idxs,4)
                data[i,idxs[0],316]=0
                data[i,idxs[1],316]=0
                data[i,idxs[2],316]=0
                data[i,idxs[3],316]=0
                #print(mask[i,:])
                mask[i,idxs[0]]=0
                mask[i,idxs[1]]=0
                mask[i,idxs[2]]=0
                mask[i,idxs[3]]=0
                pids=pids + 4
                break;
        if(missingRate>=30):
            if(len(idxs)>2):
                idxs=np.random.choice(idxs,3)
                data[i,idxs[0],316]=0
                data[i,idxs[1],316]=0
                data[i,idxs[2],316]=0
                #print(mask[i,:])
                mask[i,idxs[0]]=0
                mask[i,idxs[1]]=0
                mask[i,idxs[2]]=0
                pids=pids + 3
                break;
        if(missingRate>=20):
            if(len(idxs)>1):
                idxs=np.random.choice(idxs,2)
                data[i,idxs[0],316]=0
                data[i,idxs[1],316]=0
                #print(mask[i,:])
                mask[i,idxs[0]]=0
                mask[i,idxs[1]]=0
                pids=pids + 2
                break;
        if(missingRate>=10):
            if(len(idxs)>0):
                idxs=np.random.choice(idxs,1)
                data[i,idxs,316]=0
                mask[i,idxs]=0
                pids=pids + 1

        testMask[i,:]=testMask[i,:]-mask[i,:]
        y[i,:]=y[i,:]*testMask[i,:]

    data=data[:,:,316]
    data=data.transpose()
    mask=mask.transpose()
    y=y.transpose()
    testMask=testMask.transpose()
    data[data==0]=np.nan
    data=pd.DataFrame(data)

    data_complete=data.fillna(data.mean())

    data=data.fillna(0)
    data_complete=data_complete.fillna(0)

    data = data.to_numpy()
    data_complete = data_complete.to_numpy()

    data_complete=np.multiply(data_complete,testMask)

    #print(data_complete[0,:])
    #print(y[0,:])
    #print(testMask[0,:])
    mae = np.abs(data_complete - y).sum() / (1e-5 + np.sum(testMask))
    mre = np.abs(data_complete - y).sum() / (1e-5 + np.sum(np.abs(y)))

    outBmi=data_complete[np.nonzero(testMask)]
    inBmi=y[np.nonzero(testMask)]
#     with open('results/mean/'+str(m) +'/outBmiImp', 'wb') as fp:
#         pickle.dump(outBmi, fp)
#     with open('results/mean/'+str(m) +'/inBmiImp', 'wb') as fp:
#         pickle.dump(inBmi, fp)
    #print(len(inBmi))
    #print(len(outBmi))
    #plotBmi(inBmi,outBmi,len(inBmi))

    print("Mean MAE: ",mae)
    #print("Mean MRE: ",mre)

In [36]:
#MEAN PRED ALGO
if args.pred:
    if args.air:
        data=pd.read_csv('.../aaai/data/air/preprocess/airTest.csv',header=0)
        mask=pd.read_csv('.../aaai/data/air/preprocess/airTestMask.csv',header=0)
        data=data[['Date', 'Time', 'Month', 'PT08.S1(CO)', 'CO(GT)', 'NMHC(GT)','C6H6(GT)', 'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)','PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH']]
        del data['Time']
        del data['Date']
        del mask['Time']
        del mask['Date']
        pred(data,mask,args.pred_len)

    if args.mimic:
        data=pd.read_csv('.../aaai/data/mimic/preprocess/mimicTest.csv',header=0)
        mask=pd.read_csv('.../aaai/data/mimic/preprocess/mimicTestMask.csv',header=0)
        del data['subject_id']
        del data['charttime']
        del mask['subject_id']
        del mask['charttime']
        print(data.shape)
        data=data[['ALBUMIN', 'ANION GAP','WBC', 'BANDS', 'BICARBONATE', 'BILIRUBIN', 'BUN',
       'CHLORIDE', 'CREATININE', 'GLUCOSE', 'HEMATOCRIT', 'HEMOGLOBIN', 'INR',
       'LACTATE', 'PaCO2', 'PLATELET', 'POTASSIUM', 'PT', 'PTT', 'SODIUM'
       ]]
        pred(data,mask,args.pred_len)
        
    if args.ehr:
        ehrPred(args.pred_len)




CI 3.7765587480682647
Mean MAE:  70.89112974182785
Mean MRE:  0.9999999998892768


In [ ]:
if args.imp:
    if args.air:
        data=pd.read_csv('.../aaai/data/air/preprocess/airTest.csv',header=0)
        mask=pd.read_csv('.../aaai/data/air/preprocess/airTestMask.csv',header=0)
        data=data[['Date', 'Time', 'Month', 'PT08.S1(CO)', 'CO(GT)', 'NMHC(GT)','C6H6(GT)', 'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)','PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH']]
        del data['Time']
        del data['Date']
        del mask['Time']
        del mask['Date']
        imp(data,mask,args.missingRate)

    if args.mimic:
        data=pd.read_csv('.../aaai/data/mimic/preprocess/mimicTest.csv',header=0)
        mask=pd.read_csv('.../aaai/data/mimic/preprocess/mimicTestMask.csv',header=0)
        del data['subject_id']
        del data['charttime']
        del mask['subject_id']
        del mask['charttime']
        print(data.shape)
        data=data[['ALBUMIN', 'ANION GAP','WBC', 'BANDS', 'BICARBONATE', 'BILIRUBIN', 'BUN',
       'CHLORIDE', 'CREATININE', 'GLUCOSE', 'HEMATOCRIT', 'HEMOGLOBIN', 'INR',
       'LACTATE', 'PaCO2', 'PLATELET', 'POTASSIUM', 'PT', 'PTT', 'SODIUM'
       ]]
        imp(data,mask,args.missingRate)
        
    if args.ehr:
        ehrImpute(args.missingRate)
